In [127]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [128]:
class TokenGameState:
    def __init__(self, tokens, maximizingPlayer):
        self._tokens = tokens
        self._maximizingPlayer = maximizingPlayer

    def getTokens(self):
        return self._tokens
    def getMaximizingPlayer(self):
        return self._maximizingPlayer

In [151]:
class TokenGameHandler:

    def __init__(self,k):
        self._k = k

    def getK(self):
        return self._k

    def isWinning(self, tokens):
        if (tokens-1)%(self.getK()+1)==0:
            return False
        else:
            return True

    def getOffsprings(self, state, maximizingPlayer):
        offsprings = []
        for i in range(self.getK()):
            if state.getTokens()-i < 1:
                continue
            offsprings.append(TokenGameState(state.getTokens()-i, maximizingPlayer))
        return offsprings

    def evaluate(self, state):
        if self.isWinning(state.getTokens()):
            if state.getMaximizingPlayer():
                return 1
            else:
                return -1
        else:
            if state.getMaximizingPlayer():
                return -1
            else:
                return 1

    def isEndState(self,state):
        return state.getTokens() !=1



In [152]:
def isWinning(tokens,k):
    if (tokens-1)%(k+1)==0:
        return False
    else:
        return True

In [153]:
def evaluateState(tokens, k, maximizingPlayer):
    if isWinning(tokens,k):
        if maximizingPlayer:
            return 1
        else:
            return -1
    else:
        if maximizingPlayer:
            return -1
        else:
            return 1

In [154]:
# def minimaxAlfaBeta(tokens, k, depth, alpha, beta, maximizingPlayer):
#     if depth == 0 or tokens == 1:
#         return evaluateState(tokens, k, maximizingPlayer)
#     if maximizingPlayer:
#         for i in range(k):
#             if tokens - i < 1:
#                 continue
#             alpha = max(alpha, minimaxAlfaBeta(tokens-i, k, depth-1, alpha, beta, False))
#             if alpha >= beta:
#                 return beta
#             return alpha
#     else:
#         for i in range(k):
#             if tokens - i < 1:
#                 continue
#             beta = min(beta, minimaxAlfaBeta(tokens-i, k, depth-1, alpha, beta, True))
#             if alpha >= beta:
#                 return alpha
#             return beta


In [155]:
# print(minimaxAlfaBeta(15,2,4,-10,10,True))

1


In [156]:
def minimaxAlfaBeta2(state, depth, alpha, beta, maximizingPlayer, gameHandler):
    if depth == 0 or gameHandler.isEndState(state):
        return gameHandler.evaluate(state)
    if maximizingPlayer:
        for offspring in gameHandler.getOffsprings(state, maximizingPlayer):
            alpha = max(alpha, minimaxAlfaBeta2(offspring, depth-1, alpha, beta, False, gameHandler))
            if alpha >= beta:
                return beta
            return alpha
    else:
        for offspring in gameHandler.getOffsprings(state, maximizingPlayer):
            beta = min(beta, minimaxAlfaBeta2(offspring, depth-1, alpha, beta, True, gameHandler))
            if alpha >= beta:
                return alpha
            return beta

In [164]:
tokens = 16
k = 4
depth = 4
state = TokenGameState(tokens, True)
tokenGameHandler = TokenGameHandler(k)
print(minimaxAlfaBeta2(state,depth,-10,10,True, tokenGameHandler))

-1
